In [12]:
%load_ext autoreload
%autoreload 2

import os

from TaylorF2 import TaylorF2
import numpy as np
import torch
import torchvision.datasets as dset
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam , RMSprop

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
pyro.enable_validation(True)
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)
# Enable smoke test - run§§ the notebook cells on CI.
smoke_test = 'CI' in os.environ

In [14]:
n_train = 20000
n_test = 2000

q_train = np.random.uniform(1.0,2.0,n_train)
m_chirp_train = np.random.uniform(1,2, n_train)
chi1_train = np.random.uniform(-1.,1.,n_train)
chi2_train = np.zeros(n_train)

y_train = np.vstack((q_train,m_chirp_train , chi1_train ,chi2_train ))

q_test = np.random.uniform(1.0,5.0,n_test)
m_chirp_test = np.random.uniform(1,5, n_test)
chi1_test = np.random.uniform(-1.,1.,n_test)
chi2_test = np.zeros(n_test)
y_test = np.vstack((q_test, m_chirp_test , chi1_test ,chi2_test))

In [15]:
def get_component_from_chirp(M_chirp,q):
    """Convert chirp mass and mass ratio to component masses"""
    m2 = ((((q+1)**(1./5.))/(q**(3./5.)))*M_chirp)
    m1 = q*m2
    return m1,m2

#chi2 = 0.0 # spin zero
#Mchir = 1.0 # chirp mass

n_points = 200
rescaling =  12.5 # Neural networks seem to work better with normalized data

Mf_max = 1.0/(6**(3.0/2.0) * np.pi) # cut at ISCO
Mf_array = np.linspace(1e-3, Mf_max, n_points) # Geometric frequency
A0 = 1
x_train = np.empty((n_train, n_points))
for i,(q,Mchir,chi1,chi2) in enumerate(y_train.transpose()):
    m1,m2 = get_component_from_chirp(Mchir,q)
    pn = TaylorF2(m1, m2, chi1, chi2)
    x_train[i] = pn.compute_strain(Mf_array, Mf0=0.01 , A0 =1 )/rescaling
    
x_test = np.empty((n_test, n_points))
for i,(q,Mchir,chi1,chi2) in enumerate(y_test.transpose()):
    m1,m2 = get_component_from_chirp(Mchir,q)
    pn = TaylorF2(m1, m2, chi1, chi2)
    x_test[i] = pn.compute_strain(Mf_array, Mf0=0.01 , A0 =1)/rescaling

/Users/Rhys/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: ComplexWarning: Casting complex values to real discards the imaginary part
/Users/Rhys/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: ComplexWarning: Casting complex values to real discards the imaginary part


In [16]:
np.savetxt('./train_set.txt' , x_train)
np.savetxt('./test_set.txt' , x_test)